Team Charter US6

**Objective**: Analyze 6 years of transaction data from `../data/ROSA_financial_transactions.csv`

We will explore:
- Monthly & yearly trends
- Top customers by income and spending
- Interactive charts

In [13]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load data
df = pd.read_csv("../data/ROSA_financial_transactions.csv")
df['date'] = pd.to_datetime(df['date'])
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

# Extract ONLY the month number (01 to 12)
df['month'] = df['date'].dt.month.astype(str).str.zfill(2)  # "01", "02", ..., "12"

# Optional: If you want Year + Month as "2020-01"
df['year_month'] = df['date'].dt.strftime('%Y-%m')

df['year'] = df['date'].dt.strftime('%Y')

df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

print(f"Loaded {len(df):,} transactions from {df['date'].min().year} to {df['date'].max().year}")
df[['date', 'month', 'year_month','amount']].head(10)

Loaded 100,000 transactions from 2018 to 2023


,date,month,year_month,amount
0,2020-10-26,10,2020-10,6478.39
1,2020-01-08,01,2020-01,1255.95
2,2019-09-02,09,2019-09,7969.68
3,2020-12-02,12,2020-12,2927.41
4,2020-12-02,12,2020-12,4661.88
5,2021-04-25,04,2021-04,3649.68
6,2020-06-19,06,2020-06,504.24
7,2022-09-08,09,2022-09,8088.01
8,2022-01-30,01,2022-01,3006.30
9,2020-08-20,08,2020-08,1008.62


## Monthly & Yearly Summary

In [14]:
df['income'] = df.apply(lambda x: x['amount'] if x['type'] == 'credit' else 0, axis=1)
df['expense'] = df.apply(lambda x: x['amount'] if x['type'] in ['debit', 'transfer'] else 0, axis=1)

monthly = df.groupby('year_month').agg(Income=('income','sum'), Expense=('expense','sum')).round(2)
monthly['Net'] = monthly['Income'] - monthly['Expense']
monthly = monthly.reset_index()

yearly = df.groupby('year').agg(Income=('income','sum'), Expense=('expense','sum')).round(2)
yearly['Net'] = yearly['Income'] - yearly['Expense']
yearly = yearly.reset_index()

monthly.head(10)

,year_month,Income,Expense,Net
0,2018-03,403024.06,1008067.98,-605043.92
1,2018-04,2590313.16,5575605.31,-2985292.15
2,2018-05,2963157.71,5794762.98,-2831605.27
3,2018-06,2631600.42,5597721.48,-2966121.06
4,2018-07,2961091.68,5756808.26,-2795716.58
5,2018-08,2830015.18,5828017.08,-2998001.90
6,2018-09,2660105.18,5735058.05,-3074952.87
7,2018-10,2807436.03,5488568.67,-2681132.64
8,2018-11,2602184.49,5624660.85,-3022476.36
9,2018-12,2654333.40,5701010.24,-3046676.84


## Interactive Charts

In [15]:
fig = go.Figure()
fig.add_trace(go.Bar(x=monthly['year_month'], y=monthly['Income'], name='Income', marker_color='#32B1CD'))
fig.add_trace(go.Bar(x=monthly['year_month'], y=-monthly['Expense'], name='Expense', marker_color='#ff6b6b'))
fig.add_trace(go.Scatter(x=monthly['year_month'], y=monthly['Net'], mode='lines+markers', name='Net', line=dict(color='white', width=4)))
fig.update_layout(title="Monthly Trend", height=600, barmode='relative')
fig.show()

In [21]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots: 2 rows, 1 column (stacked vertically)
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Top 15 Income Generators", "Top 15 Biggest Spenders"),
    vertical_spacing=0.12,
    row_heights=[0.5, 0.5]
)

# Top 15 Credit (Income) - Horizontal bar
fig.add_trace(
    go.Bar(
        y=top_credit['customer_id'].astype(str),
        x=top_credit['total_credit'],
        orientation='h',
        name="Income",
        marker_color='#32B1CD',
        text=top_credit['total_credit'].apply(lambda x: f"${x:,.0f}"),
        textposition="outside",
        hovertemplate="<b>Customer %{y}</b><br>Income: $%{x:,.0f}<extra></extra>"
    ),
    row=1, col=1
)

# Top 15 Debit (Spending) - Horizontal bar
fig.add_trace(
    go.Bar(
        y=top_debit['customer_id'].astype(str),
        x=top_debit['total_debit'],
        orientation='h',
        name="Spending",
        marker_color='#ff6b6b',
        text=top_debit['total_debit'].apply(lambda x: f"${x:,.0f}"),
        textposition="outside",
        hovertemplate="<b>Customer %{y}</b><br>Spending: $%{x:,.0f}<extra></extra>"
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    title_text="Top 15 Customers Analysis",
    title_x=0.5,
    height=900,  # taller for vertical stack
    showlegend=False,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color="white")
)

# Reverse y-axes so highest is at the top
fig.update_yaxes(autorange="reversed", row=1, col=1)
fig.update_yaxes(autorange="reversed", row=2, col=1)

# Axis labels
fig.update_xaxes(title_text="Income ($)", row=1, col=1)
fig.update_xaxes(title_text="Spending ($)", row=2, col=1)

fig.show()

# **Financial Dashboard – Key Insights (2018–2023)**

## **Executive Summary**
Over the 6-year analysis period:

- **Total unique customers:** 1,000  
- **Overall trend:** The company is running at a significant net loss every single month.  
- **Average monthly income:** ~$2.6M – $2.9M  
- **Average monthly expense:** ~$5.5M – $5.8M  
- **Average monthly net loss:** **-$2.8M to -$3.2M**  
- **Cumulative loss:** Exceeds **-$200M**

---

## **1. Monthly Trend (6 Years)**  
### **Key Observations**
- Expenses are consistently **almost double** the income.  
- **No month** has ever been profitable.  
- Loss patterns are **extremely stable year after year**, suggesting a **structural issue**, not a seasonal one.

---

## **2. Top 15 Income Generators vs Top 15 Biggest Spenders**

### **Top Customers Comparison**

#### **Income Side (Credit)**
- Best customer (**#499**) generated **$317,201** total credit.  
- Even top earners contribute relatively little compared to top spenders.

#### **Expense Side (Debit/Transfers)**
- Worst spender (**#30**) generated **$482,399** in expenses.  
- The top 15 spenders alone account for **millions in outflows**.

### **Critical Insight**
There is **almost no overlap** between top revenue-generating customers and top spenders.  
This indicates potential issues in:

- Customer acquisition cost  
- Pricing model  
- Fraud / bad debt risk

---

## **Recommendations for Immediate Action**

- **Urgent profitability review** — business model appears unsustainable.  
- Analyze why **high-spending customers do not generate matching revenue**.  
- Segment customers by **net contribution** (income minus expenses).  
- Update **credit policies** or implement **customer risk scoring**.  
- Investigate **possible fraud patterns** among largest negative-balance accounts.

---

## **Conclusion**
Despite high transaction volume, the company is **hemorrhaging money every month**.  
There is **no indication of improvement** over the entire 6-year period.  
Immediate structural and financial intervention is required.
